In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import datetime
# import cmocean.cm as cmo
import glob

matplotlib.rcParams['font.serif'] = "Times New Roman"
matplotlib.rcParams['font.family'] = "serif"
matplotlib.rcParams['font.size'] = '15.0'
matplotlib.rcParams['text.usetex'] = 'True'

In [2]:
# get sample volumes per file
infile = '../data/IFCB_DATA/sample_volumes.csv'
ifcb_vols = pd.read_csv(infile)
ifcb_vols.columns

Index([u'Unnamed: 0', u'orig_id', u'runTime', u'inhibitTime', u'vol_ml'], dtype='object')

In [3]:
#load in the ecotaxa counts

infile = '../data/EcoTaxa_output/ecotaxa_export_2840_20210128_1745.tsv'
data = pd.read_csv(infile, sep = '\t', parse_dates = ['object_date'])
data = data[data['object_date'].dt.year >2019]


/Users/sclayton/opt/anaconda3/envs/soph_py2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
# reshape the data into a table with numbers of each taxa for each sample
# get list of all taxa in table
taxa = pd.unique(data.object_annotation_category)

# initialise an empty dataframe to fill with taxa abundance for each sample
df = pd.DataFrame(columns = taxa)
df['sample_id'] = pd.unique(data.sample_id)
df['date'] = ""
df['vol_ml'] = 0 

for s in pd.unique(data.sample_id):
    df.loc[df.sample_id == s,['date']] = pd.unique(data.object_date.dt.date)[0]
    df.loc[df.sample_id == s,['vol_ml']] = np.float(ifcb_vols.loc[ifcb_vols.orig_id == s, ['vol_ml']].values)

    for t in taxa:
        sub = data[(data.object_annotation_category == t) & (data.sample_id == s)]

        if (sub[sub.object_annotation_category == t]).empty:
            test = 0
        else:
            test = len((sub[(sub.object_annotation_category == t)]))
        df.loc[df.sample_id == s, [t]] = test

datetime_index = pd.DatetimeIndex(df.date.values)
df = df.set_index(datetime_index)
    
# mm_pm = pm_df.resample('D').mean().dropna()
# ss_pm = pm_df.resample('D').std().dropna()

In [14]:
# df.to_csv('../data/taxa_abundance_v0.1.csv', sep=',', index= False)